第一次参加

In [ ]:
import megengine as mge
import megengine.module as M
import megengine.functional as F

patchsz = 256
batchsz = 16

In [ ]:
class Predictor(M.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = M.Sequential(
            M.Conv2d(4, 32, 3, padding = 1, bias = True),
            M.PReLU(),
        )
        self.downsample1 = M.Conv2d(32, 32, 2, stride=2, padding=0)
        self.conv0 = M.Sequential(
            M.Conv2d(32, 32, 3, padding = 1, bias = True),
            M.PReLU(),
        )
        self.downsample2 = M.Conv2d(32, 32, 2, stride=2, padding=0)
        self.upda = M.Conv2d(32, 32, 3, padding = 1, bias = True)
        self.convx = M.Sequential(
            M.Conv2d(64, 4, 3, padding = 1, bias = True),
            M.PReLU(),
        )
        self.conv2 = M.Sequential(
            M.Conv2d(4, 32, 3, padding = 1, bias = True),
            M.PReLU(),
            M.Conv2d(32, 32, 3, padding = 1, bias = True),
            M.PReLU(),
        )
        self.conv3 = M.Sequential(
            M.BatchNorm2d(32),
            M.PReLU(),
            M.Conv2d(32, 32, 3, padding = 1, bias = True),
            M.BatchNorm2d(32),
            M.PReLU(),
            M.Conv2d(32, 32, 3, padding = 1, bias = True),           
        )
        self.conv4 = M.Sequential(
            M.Conv2d(32, 32, 3, padding = 1, bias = True),
            M.PReLU(),
            M.Conv2d(32, 4, 3, padding = 1, bias = True),
            M.PReLU(),
        )      
        self.upsample2 = M.ConvTranspose2d(64, 64, kernel_size=2, stride=2, bias=True)
        self.upsample1 = M.ConvTranspose2d(32, 32, kernel_size=2, stride=2, bias=True)
    def forward(self, x):
        n, c, h, w = x.shape
        x = x.reshape((n, c, h // 2, 2, w // 2, 2)).transpose((0, 1, 3, 5, 2, 4)).reshape((n, c * 4, h // 2, w // 2))
        a = x
        x = self.downsample1(self.conv1(x))
        b = x
        x = self.conv0(x)
        x = self.downsample2(x)
        x = self.upda(x)
        x = self.upsample1(x)
        x = F.concat([b,x],1)
        x = self.upsample2(x)
        x = self.convx(x)
        x = a+x
        x = self.conv2(x)
        x = self.conv3(x)+x
        x = self.conv4(x)
        x = x.reshape((n, c, 2, 2, h // 2, w // 2)).transpose((0, 1, 4, 2, 5, 3)).reshape((n, c, h, w))
        return x

In [ ]:
if __name__ == '__main__':
    
    import random
    import numpy as np
    from megengine.utils.module_stats import module_stats

    net = Predictor()

    input_data = np.random.rand(1, 1, 256, 256).astype("float32")
    total_stats, stats_details = module_stats(
        net,
        inputs = (input_data,),
        cal_params = True,
        cal_flops = True,
        logging_to_stdout = True,
    )

    print("params %.3fK MAC/pixel %.0f"%(total_stats.param_dims/1e3, total_stats.flops/input_data.shape[2]/input_data.shape[3]))


In [ ]:
import numpy as np
print('loading data')
content = open('dataset/burst_raw/competition_train_input.0.2.bin', 'rb').read()
samples_ref = np.frombuffer(content, dtype = 'uint16').reshape((-1,256,256))
content = open('dataset/burst_raw/competition_train_gt.0.2.bin', 'rb').read()
samples_gt = np.frombuffer(content, dtype = 'uint16').reshape((-1,256,256))

In [ ]:
import megengine.data as data
import megengine.data.transform as T

transform = T.Compose([
    T.Resize(256),
    T.CenterCrop(256),
    T.RandomHorizontalFlip(),
   # T.ColorJitter(brightness=0.5, contrast=0.5, saturation=0, hue=0),
    T.ToMode("CHW"),
])
#processed_img = transform.apply(samples)  # Still NumPy ndarray here

In [ ]:
def train_one_epoch(epoch,net,samples_ref,samples_gt,batchsz,patchsz,opt,gm,lr):
    net.train()
    losses = []
    train_steps = int(len(samples_ref)/batchsz)
    idx = np.arange(train_steps)
    np.random.shuffle(idx)
    samples_ref = samples_ref[idx]
    samples_gt = samples_gt[idx]
    
    for it in range(0, train_steps):

        opt.zero_grad()
        batch_inp_np = np.zeros((batchsz, 1, patchsz, patchsz), dtype = 'float32')
        batch_out_np = np.zeros((batchsz, 1, patchsz, patchsz), dtype = 'float32')

        for i in range(batchsz):
            batch_inp_np[i, 0, :, :] = np.float32(train_samples_ref[it*batchsz+i, :, :]) * np.float32(1 / 65536)
            batch_out_np[i, 0, :, :] = np.float32(train_samples_gt[it*batchsz+i, :, :]) * np.float32(1 / 65536)
            
        batch_inp = mge.tensor(batch_inp_np)
        batch_out = mge.tensor(batch_out_np)

        with gm:
            pred = net(batch_inp)
            loss = F.abs(pred - batch_out).mean()
            gm.backward(loss)
            opt.step().clear_grad()

        loss = float(loss.numpy())
        losses.append(loss)
    print(f'epoch:{epoch} train_loss:{np.array(losses).mean()}')
    return net,losses

def eval_one_epoch(epoch,net,samples_ref,samples_gt,batchsz,patchsz):
    net.eval()
    losses = []
    steps = int(len(samples_ref)/batchsz)
    
    for it in range(0, steps):

        batch_inp_np = np.zeros((batchsz, 1, patchsz, patchsz), dtype = 'float32')
        batch_out_np = np.zeros((batchsz, 1, patchsz, patchsz), dtype = 'float32')

        for i in range(batchsz):
            batch_inp_np[i, 0, :, :] = np.float32(train_samples_ref[it*batchsz+i, :, :]) * np.float32(1 / 65536)
            batch_out_np[i, 0, :, :] = np.float32(train_samples_gt[it*batchsz+i, :, :]) * np.float32(1 / 65536)

        batch_inp = mge.tensor(batch_inp_np)
        batch_out = mge.tensor(batch_out_np)

        with gm:
            pred = net(batch_inp)
            loss = F.abs(pred - batch_out).mean()
        loss = float(loss.numpy())
        losses.append(loss)
    print(f'epoch:{epoch} val_loss:{np.array(losses).mean()}')
    return losses


In [ ]:
train_samples_ref = samples_ref[:]
train_samples_gt = samples_gt[:]
val_samples_ref = samples_ref[:800]
val_samples_gt = samples_gt[:800]

lr = 1e-3
opt = mge.optimizer.Adam(net.parameters(), lr = lr)
gm = mge.autodiff.GradManager().attach(net.parameters())
train_losses = []
val_losses = []


epochs =20
print('start to train')
for epoch in range(epochs):

    net,losses = train_one_epoch(epoch,net,train_samples_ref,train_samples_gt,batchsz,patchsz,opt,gm,lr)
    train_losses.append(losses)
    losses = eval_one_epoch(epoch,net,val_samples_ref,val_samples_gt,batchsz,patchsz)
    val_losses.append(losses)

fout = open('model','wb')
mge.save(net.state_dict(), fout)
fout.close()

In [1]:
print('prediction')

content = open('dataset/burst_raw/competition_test_input.0.2.bin', 'rb').read()
samples_ref = np.frombuffer(content, dtype = 'uint16').reshape((-1,256,256))
fout = open('workspace/competition_prediction.new4.bin', 'wb')

import tqdm
net.eval()
for i in tqdm.tqdm(range(0, len(samples_ref), batchsz)):
    
    i_end = min(i + batchsz, len(samples_ref))
    batch_inp = mge.tensor(np.float32(samples_ref[i:i_end, None, :, :]) * np.float32(1 / 65536))
    pred = net(batch_inp)
    pred = (pred.numpy()[:, 0, :, :] * 65536).clip(0, 65535).astype('uint16')
    fout.write(pred.tobytes())

fout.close()

prediction


NameError: name 'np' is not defined

In [ ]:
import numpy as np
#content = open('dataset/burst_raw/competition_test_input.0.2.bin', 'rb').read()
#samples_ref = np.frombuffer(content, dtype = 'uint16').reshape((-1,256,256))
content = open('workspace/competition_prediction.new4.bin', 'rb').read()
samples_ref = np.frombuffer(content, dtype = 'uint16').reshape((-1,256,256))
content = open('workspace/competition_prediction.new3.bin', 'rb').read()
samples_gt = np.frombuffer(content, dtype = 'uint16').reshape((-1,256,256))
from matplotlib import pyplot as plt
for i in range(100):
    fig = plt.figure()
    plt.subplot(1, 2, 1)
    plt.imshow(samples_ref[i],cmap='gray')
    plt.subplot(1, 2, 2)
    plt.imshow(samples_gt[i],cmap='gray')
    plt.show()